
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/05_Top_p_Top_k.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🧠 Top-k vs Top-p: Jak myśli AI?

Mamy zdanie: **"Na obiad zjem..."**
Model wyliczył prawdopodobieństwa dla następnego słowa. Zobaczmy, jak filtry **Top-k** i **Top-p** zmieniają pulę dostępnych opcji.

*   **Top-k:** Bierze sztywną liczbę kandydatów.
*   **Top-p:** Bierze kandydatów do momentu osiągnięcia progu pewności (skumulowane prawdopodobieństwo).

In [1]:
import pandas as pd
import numpy as np

# 1. SYMULACJA MÓZGU AI
# Model przewiduje, co będzie po: "Na obiad zjem..."
# Mamy słownik: {Słowo: Prawdopodobieństwo}
kandydaci = {
    'pizzę': 0.50,      # Bardzo prawdopodobne
    'zupę': 0.20,
    'schabowego': 0.15,
    'kanapkę': 0.09,
    'gwoździe': 0.05,   # Mało sensowne
    'chmurę': 0.01      # Totalna bzdura
}

# Zamieniamy na tabelę
df = pd.DataFrame(list(kandydaci.items()), columns=['Słowo', 'Prawdopodobieństwo'])
print("--- WSZYSTKIE OPCJE MODELU (RAW) ---")
display(df)
print("\n")


# --- METODA 1: TOP-K (Sztywna granica) ---
K = 3
print(f"--- ✂️ FILTR TOP-K (k={K}) ---")
print(f"Zasada: Bierzemy tylko {K} najlepszych.")

df_top_k = df.head(K).copy()
print(f"Dostępne słowa: {df_top_k['Słowo'].tolist()}")
print("Reszta została odcięta (nawet jeśli 'kanapka' była sensowna).")
print("\n")


# --- METODA 2: TOP-P (Elastyczna granica) ---
P = 0.90  # 90%
print(f"--- 🧠 FILTR TOP-P (p={P}) ---")
print(f"Zasada: Zbieraj słowa, aż uzbierasz {P*100}% pewności.")

df['Suma_Skumulowana'] = df['Prawdopodobieństwo'].cumsum()

# Wybieramy słowa, które łapią się w progu 0.90
# (Trick: bierzemy te, gdzie suma poprzednich była mniejsza niż P)
df_top_p = df[df['Suma_Skumulowana'] - df['Prawdopodobieństwo'] < P].copy()

print(df_top_p[['Słowo', 'Prawdopodobieństwo', 'Suma_Skumulowana']])
print(f"\nDostępne słowa: {df_top_p['Słowo'].tolist()}")
print(f"Zauważ: Top-p wziął 'kanapkę' (bo mieściła się w 90%), ale odrzucił 'gwoździe'.")

--- WSZYSTKIE OPCJE MODELU (RAW) ---


,Słowo,Prawdopodobieństwo
0,pizzę,0.50
1,zupę,0.20
2,schabowego,0.15
3,kanapkę,0.09
4,gwoździe,0.05
5,chmurę,0.01




--- ✂️ FILTR TOP-K (k=3) ---
Zasada: Bierzemy tylko 3 najlepszych.
Dostępne słowa: ['pizzę', 'zupę', 'schabowego']
Reszta została odcięta (nawet jeśli 'kanapka' była sensowna).


--- 🧠 FILTR TOP-P (p=0.9) ---
Zasada: Zbieraj słowa, aż uzbierasz 90.0% pewności.
        Słowo  Prawdopodobieństwo  Suma_Skumulowana
0       pizzę                0.50              0.50
1        zupę                0.20              0.70
2  schabowego                0.15              0.85
3     kanapkę                0.09              0.94

Dostępne słowa: ['pizzę', 'zupę', 'schabowego', 'kanapkę']
Zauważ: Top-p wziął 'kanapkę' (bo mieściła się w 90%), ale odrzucił 'gwoździe'.
